In [1]:
# lib standart
import numpy as np 
import pandas as pd
import re
from sklearn.model_selection import train_test_split # Untuk split data
from sklearn.neural_network import MLPClassifier # Untuk Algoritma ML yang akan di pakai
from sklearn.model_selection import GridSearchCV  # untuk tuning hyperparameter
from sklearn.pipeline import Pipeline # untuk membangun pipeline ML
from sklearn.compose import ColumnTransformer # bagian dari pipe line untuk handling kolom 
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report # menghitung nilai f1
from sklearn.feature_extraction.text import CountVectorizer
import pickle # menyimpan model
import text_preprocessing as tp

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\666Rakels\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
data_train = pd.read_csv('train_preprocess.tsv', sep='\t',header = None)
data_train

,0,1
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [3]:
df_data = data_train.rename(columns={0: 'Text', 1: 'Sentimen'})
df_data.head()

,Text,Sentimen
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


### Normalisasi Data

In [27]:
import pandas as pd
import text_preprocessing as tp
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

#load train dataset
df_train = pd.read_csv('train_preprocess.tsv', sep='\t',header = None)
#df_train = pd.read_csv('/content/train_preprocess.tsv', sep='\t', header=None)
df_data = df_train.rename(columns={0: 'Text',1: 'Sentimen'})

#load kamus alay dataset
kamus_alay = pd.read_csv('new_kamusalay.csv', encoding='latin-1', header=None)
#kamus_alay = pd.read_csv('/content/new_kamusalay.csv', encoding='latin-1', header=None)
kamus_alay = kamus_alay.rename(columns={0: 'original', 1: 'replacement'})

id_stopword_dict = pd.read_csv('stopwordbahasa.csv', header=None)
#id_stopword_dict = pd.read_csv('/content/stopwordbahasa.csv', header=None)
id_stopword_dict = id_stopword_dict.rename(columns={0: 'stopword'})


# Define the preprocessing function
#text cleansing with regex
def lowercase(text):
    return text.lower()

def hapus_karakter_ga_penting(text):
    text = re.sub('\n',' ',text) # Hapus enter
    text = re.sub('nya|deh|sih',' ',text) # Hapus stopwords tambahan
    text = re.sub('RT',' ',text) # Hapus RT
    text = re.sub('USER',' ',text) # Hapus USER
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', ' ',text) # Hapus URL
    text = re.sub('  +', ' ', text) # Hapus extra spaces
    text = re.sub('[^a-zA-Z0-9]', ' ', text) #Hapus non huruf dan angka  
    text = re.sub('\@[a-zA-Z0-9]*', ' ', text) #Hapus non huruf atau apostrophe
    text = ' '.join([w for w in text.split() if len(w)>1]) #Hapus huruf tunggal 
    return text
    
def hapus_nonhurufangka(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

df_data_map = dict(zip(df_data['Text'], df_data['Sentimen']))
def normalisasi_alay(text):
    return ' '.join([df_data_map[word] if word in df_data_map else word for word in text.split(' ')])

def hapus_stopword(text):
    text = ' '.join(['' if word in id_stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Hapus extra spaces
    text = text.strip()
    return text

def stemming(text):
    return stemmer.stem(text)

def cleansing(text):
    text = lowercase(text) 
    text = hapus_nonhurufangka(text) 
    text = hapus_karakter_ga_penting(text) 
    text = normalisasi_alay(text) 
    text = stemming(text) 
    text = hapus_stopword(text) 
    return text


# Add the new column to the DataFrame
df_data['Text_baru'] = df_data['Text'].apply(cleansing)

# Create the final DataFrame with the original columns
data_train = df_data[['Text', 'Sentimen']].assign(Text_baru=df_data['Text_baru'])

# Display the result
print(df_data)


                                                    Text  Sentimen  \
0      warung ini dimiliki oleh pengusaha pabrik tahu...  positive   
1      mohon ulama lurus dan k212 mmbri hujjah partai...   neutral   
2      lokasi strategis di jalan sumatera bandung . t...  positive   
3      betapa bahagia nya diri ini saat unboxing pake...  positive   
4      duh . jadi mahasiswa jangan sombong dong . kas...  negative   
...                                                  ...       ...   
10995                                       tidak kecewa  positive   
10996  enak rasa masakan nya apalagi kepiting yang me...  positive   
10997        hormati partai-partai yang telah berkoalisi   neutral   
10998  pagi pagi di tol pasteur sudah macet parah , b...  negative   
10999  meskipun sering belanja ke yogya di riau junct...  positive   

                                               Text_baru  
0      warung milik usaha pabrik puluh kenal putih ba...  
1      mohon ulama lurus k212 mmbri hujja

In [5]:
import pandas as pd

category = pd.get_dummies(data_train.Sentimen)
df_baru = pd.concat([data_train, category], axis=1)
df_baru = df_baru.drop(columns='Sentimen')

df_baru['Text'] = df_baru['Text'].replace('\n', ' ').str.lower()

df_baru.head()

,Text,Text_baru,negative,neutral,positive
0,warung ini dimiliki oleh pengusaha pabrik tahu...,warung milik usaha pabrik puluh kenal putih ba...,0,0,1
1,mohon ulama lurus dan k212 mmbri hujjah partai...,mohon ulama lurus k212 mmbri hujjah partai diw...,0,1,0
2,lokasi strategis di jalan sumatera bandung . t...,lokasi strategis jalan sumatera bandung man ut...,0,0,1
3,betapa bahagia nya diri ini saat unboxing pake...,betapa positive unboxing paket barang positive...,0,0,1
4,duh . jadi mahasiswa jangan sombong dong . kas...,duh mahasiswa negative ka kartu kuning ajar po...,1,0,0


### Feature Extraction

melakukan feature Extration menggunakan metode Bag of Words

In [6]:
data_preprocessed = df_data.Text_baru.tolist()

In [7]:
count_vect = CountVectorizer()
    
# melakukan fitting dan transformasi pada dokumen
count_vect.fit(data_preprocessed)

# melihat hasil representasi bag of words
X = count_vect.fit_transform(data_preprocessed)

print("Feature extration selesai")

Feature extration selesai


In [8]:
# menyimpan objek count_vect 

pickle.dump(count_vect, open("feature_New3.sav", "wb"))

### Train - Test Data

In [9]:
y = data_train.Sentimen

In [10]:
classes = df_data.Sentimen

In [11]:
classes

0        positive
1         neutral
2        positive
3        positive
4        negative
           ...   
10995    positive
10996    positive
10997     neutral
10998    negative
10999    positive
Name: Sentimen, Length: 11000, dtype: object

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size = 0.20, random_state = 40)

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

model = Pipeline([('algoritma', MLPClassifier())])
parameter_grid = {
    'algoritma__hidden_layer_sizes': [(i,) for i in [1, 10, 20]],
    'algoritma__activation': ['relu', 'tanh', 'logistic'],
    'algoritma__learning_rate_init': [0.01, 0.1],
}

model_NN3 = GridSearchCV(model, parameter_grid, cv=5)



# Training (Neural Network)

In [14]:
# model_NN3 = MLPClassifier()

In [15]:
%%time
model_NN3.fit(X_train,y_train)

print('testing selesai')

d:\Binar\Data Science\Code\envbinar\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


testing selesai
CPU times: total: 19min 21s
Wall time: 19min 22s


In [16]:
model_NN3.best_params_ 

{'algoritma__activation': 'logistic',
 'algoritma__hidden_layer_sizes': (20,),
 'algoritma__learning_rate_init': 0.1}

In [17]:
pickle.dump(model_NN3, open("model_NN3.sav", 'wb'))

# Evaluasi Model

### Untuk Semua Data

In [18]:
y_pred = model_NN3.predict(X)

In [19]:
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

    negative       0.95      0.91      0.93      3436
     neutral       0.94      0.93      0.93      1148
    positive       0.96      0.98      0.97      6416

    accuracy                           0.95     11000
   macro avg       0.95      0.94      0.95     11000
weighted avg       0.95      0.95      0.95     11000



### Untuk Training

In [20]:
y_pred_train = model_NN3.predict(X_train)

In [21]:
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

    negative       1.00      0.96      0.98      2775
     neutral       1.00      1.00      1.00       915
    positive       0.98      1.00      0.99      5110

    accuracy                           0.99      8800
   macro avg       0.99      0.98      0.99      8800
weighted avg       0.99      0.99      0.99      8800



### Untuk data testing

In [22]:
y_pred_test = model_NN3.predict(X_test)

In [23]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

    negative       0.77      0.72      0.74       661
     neutral       0.71      0.64      0.67       233
    positive       0.88      0.92      0.90      1306

    accuracy                           0.83      2200
   macro avg       0.78      0.76      0.77      2200
weighted avg       0.82      0.83      0.83      2200



In [24]:
# Untuk lebih menyakinkan lagi, kita juga bisa melakukan "Cross Validation"
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

kf = KFold(n_splits=5,random_state=42,shuffle=True)

accuracies = []

y = classes

for iteration, data in enumerate(kf.split(X), start=1):

    data_train   = X[data[0]]
    target_train = y[data[0]]

    data_test    = X[data[1]]
    target_test  = y[data[1]]

    clf = MLPClassifier()
    clf.fit(data_train,target_train)

    preds = clf.predict(data_test)

    # for the current fold only    
    accuracy = accuracy_score(target_test,preds)

    print("Training ke-", iteration)
    print(classification_report(target_test,preds))
    print("======================================================")

    accuracies.append(accuracy)

# this is the average accuracy over all folds
average_accuracy = np.mean(accuracies)

print()
print()
print()
print("Rata-rata Accuracy: ", average_accuracy)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.74      0.70      0.72       680
     neutral       0.73      0.64      0.68       239
    positive       0.84      0.89      0.87      1281

    accuracy                           0.80      2200
   macro avg       0.77      0.74      0.76      2200
weighted avg       0.80      0.80      0.80      2200

Training ke- 2
              precision    recall  f1-score   support

    negative       0.70      0.72      0.71       706
     neutral       0.66      0.61      0.63       220
    positive       0.85      0.85      0.85      1274

    accuracy                           0.78      2200
   macro avg       0.74      0.73      0.73      2200
weighted avg       0.78      0.78      0.78      2200

Training ke- 3
              precision    recall  f1-score   support

    negative       0.74      0.74      0.74       682
     neutral       0.73      0.65      0.68       215
    positive       0.87      0

# Predict

In [26]:
import pickle
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
import text_preprocessing

# Download the Indonesian stopwords if not already downloaded
nltk.download('stopwords')

count_vect = pickle.load(open("feature_New3.sav", "rb"))
model_NN = pickle.load(open("model_NN3.sav", "rb"))

contoh = "warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !"
preprocessed_text = cleansing(contoh)
text_vector = count_vect.transform([preprocessed_text])

result = model_NN.predict(text_vector)[0]
print("text : ", contoh)
print("\ntext_new : ", preprocessed_text)
print("\nSentiment : ", result)



text :  warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !

text_new :  warung milik usaha pabrik puluh kenal putih bandung positive padu ahli masak padu kretivitas warung me jikan menu utama bahan menu ayam selera indonesia harga jangkau bletoka kalah asli tegal

Sentiment :  positive


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\666Rakels\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
